## test cells?


In [ ]:
const Mocha = require('mocha');

// call test runner
let mocha;
if (typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        reporter: 'list',
        reporterOptions: {
            stream: false,
        },
        //parallel: true,
        isWorker: true,
        jobs: 10,
        timeout: 10000,
    });
}

function resetTests(suite) {
    suite.tests.forEach(function (t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

function testCells(cells, ctx = {}) {
    resetTests(mocha.suite);
    if (!cells) {
        cells = 'mocha test the test cell runner';
    }
    if (!cells[0].code) {
        cells = typeof cells === 'string'
            //   multi-cell syntax           single cell syntax notebook.ipynb[cell ref]
            && (!cells.includes('.ipynb') || cells.includes('['))
            ? [importer.interpret(cells)]
            : importer.interpret(cells);
    }

    // TODO: add multi lingual support here

    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        //Object.assign(ctx, Mocha);
        // TODO: add other require statements below to interent includes?
        try {
            console.log('testing cell:', r.id)
            var required = importer.import(r.id, ctx);
            Object.assign(ctx, required);
        } catch (e) {
            console.log(e);
        }
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resolve(failures);
    }));
}

module.exports = testCells;



mocha test the test cell runner?


In [ ]:
const assert = require('assert');
const sinon = require('sinon');
const proxyquire = require('proxyquire');
const EventEmitter = require('events');

describe('testCells', function () {
  let testCells;
  let importerStub;
  let mochaRunStub;
  let suiteMock;
  let mochaMock;

  beforeEach(() => {
    suiteMock = new EventEmitter();
    suiteMock.tests = [];
    suiteMock.suites = [];
    suiteMock.emit = sinon.spy();

    mochaRunStub = sinon.stub().callsFake(cb => cb(0));

    mochaMock = function () {
      return {
        suite: suiteMock,
        run: mochaRunStub
      };
    };
    mochaMock.prototype = function () { };
    mochaMock.prototype.ui = sinon.stub();
    mochaMock.prototype.reporter = sinon.stub();
    mochaMock.prototype.timeout = sinon.stub();

    importerStub = {
      interpret: sinon.stub(),
      import: sinon.stub()
    };

    testCells = importer.import('test cells', {
      require: function (string) {
        return proxyquire(string, {
          'mocha': mochaMock,
        })
      },
      importer: importerStub,
      cached: false,
    });
  });

  it('should handle null cells input and interpret string', async () => {
    importerStub.interpret.returns({ id: 'test-id', code: 'describe("sample", () => it("passes", () => {}));' });
    importerStub.import.returns({});

    const failures = await testCells(null);
    assert.strictEqual(failures, 0);
    sinon.assert.called(importerStub.interpret);
    sinon.assert.called(importerStub.import);
  });

  it('should interpret a string path (non-ipynb)', async () => {
    importerStub.interpret.returns([{ id: 'test-id', code: 'describe("test", function () {})' }]);
    importerStub.import.returns({});

    const failures = await testCells('test.js');
    assert.strictEqual(failures, 0);
  });

  it('should wrap raw code string as an array if not array', async () => {
    importerStub.interpret.returns({ id: 'inline-code', code: 'describe("inline", function () {})' });
    importerStub.import.returns({});

    const failures = await testCells('describe("inline", function () {})');
    assert.strictEqual(failures, 0);
  });

  it('should interpret array of cell objects', async () => {
    const cells = [{ id: 'a', code: 'describe("suite", function () {})' }];
    importerStub.import.returns({});

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });

  it('should handle import error gracefully', async () => {
    const cells = [{ id: 'b', code: 'describe("suite", function () {})' }];
    importerStub.import.throws(new Error('import error'));

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });
});


## test runner?

watch files run tests?

How to use: 

```
node -e "require('./Core').import('watch files run tests')('**/zuora*.ipynb', 'zuora to eloqua.ipynb')"

node -e "require('./Core').import('watch files run tests')('**/*.ipynb', 'test test runner')"```

TODO: restart this test script every loop? fork and abandon current thread?

TODO: update cell cache in intrepret notebooks.ipynb

TODO: git apply without whitespace, reset the rest?  separate index?  how do make git-scenario app?

TODO: re-import cells on Utilities/.modules folder change. run all "watcher" commands from a test below?  mocha.grep?





In [ ]:
var chokidar = require("chokidar");
var importer = require('../Core');
var testCells = importer.import('test cells');

// TODO: code analysis to combine blocks into modules?

var rateLimiter, done = true;
function testWatcher(files, tests) {
    files = typeof files === 'string' ? [files] : files;
    console.log('watching ' + files + ' - ' + path.resolve('.'))
    var watcher = chokidar.watch(files, {
        interval: 1000,
        atomic: 1000,
        awaitWriteFinish: true
    });
    watcher.on("change", function (event, path) {
        if (!done) {
            return;
        }
        console.log('running all tests');
        done = false;
        return testCells(tests).then(() => (done = true))
    });
    testCells(tests);

    var stdin = process.openStdin();
    stdin.addListener("data", function (d) {
        stdin.close();
    });
}
module.exports = testWatcher;



test sigma index?

ROUTE = /test

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <title>Service Graph - Sigma.js</title>
  <!-- Load Sigma.js v2 -->
  <link href="/rpc/sigma-style.css" rel="stylesheet" />
  <script src="https://cdnjs.cloudflare.com/ajax/libs/graphology/0.26.0/graphology.umd.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/sigma@2.3.0/build/sigma.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/graphology-library/dist/graphology-library.min.js"></script>
</head>

<body>

  <div id="sigma-container"></div>
  <div id="info">Zoom & drag to explore</div>
  <div id="legend">
    <div class="legend-item">
      <div class="legend-color" style="background-color: #6699cc;"></div> Imports
    </div>
    <div class="legend-item">
      <div class="legend-color" style="background-color: #ff9966;"></div> Dependencies
    </div>
  </div>
  <script src="/rpc/sigma-script.js"></script>
</body>

</html>


sigma-style.css?


In [ ]:
html,
body {
  margin: 0;
  height: 100%;
  overflow: hidden;
  font-family: sans-serif;
}

#sigma-container {
  width: 100vw;
  height: 100vh;
}

#info {
  position: absolute;
  top: 10px;
  left: 10px;
  background: rgba(255, 255, 255, 0.9);
  padding: 10px;
  border-radius: 6px;
  box-shadow: 0 0 5px rgba(0, 0, 0, 0.2);
  z-index: 100;
}

#legend {
  position: absolute;
  bottom: 20px;
  left: 20px;
  background: rgba(255, 255, 255, 0.9);
  border: 1px solid #ccc;
  border-radius: 8px;
  padding: 10px;
  font-family: sans-serif;
  font-size: 14px;
}

.legend-item {
  display: flex;
  align-items: center;
  margin-bottom: 5px;
}

.legend-color {
  width: 16px;
  height: 16px;
  margin-right: 8px;
  border-radius: 3px;
}


sigma-script.js?

ROUTE = /sigma-script.js


In [ ]:
const graph = new graphology.Graph();

// Example nodes (services)
const services = [
  'Auth Service', 'User DB', 'Frontend', 'Billing Service',
  'Notification', 'Search Engine', 'Cache', 'API Gateway',
  'Logging', 'Monitoring', 'AI Recommender', 'Backup Service'
];

// Add nodes
services.forEach((name, i) => {
  graph.addNode(name, {
    label: name,
    x: Math.cos((2 * Math.PI * i) / services.length),
    y: Math.sin((2 * Math.PI * i) / services.length),
    size: 10 + Math.random() * 5,
    color: '#007acc'
  });
});

// Add random connections between services
for (let i = 0; i < services.length * 1.5; i++) {
  const source = services[Math.floor(Math.random() * services.length)];
  const target = services[Math.floor(Math.random() * services.length)];
  if (source !== target && !graph.hasEdge(source, target)) {
    graph.addEdge(source, target, {
      color: '#ccc'
    });
  }
}

const sensibleSettings = graphologyLibrary.layoutForceAtlas2.inferSettings(graph);
graphologyLibrary.layoutForceAtlas2.assign(graph, {
  iterations: 50,
  settings: sensibleSettings
});
//forceAtlas2.assign(graph, {
//  iterations: 100,
//  settings: { gravity: 1 }
//});

// Initialize Sigma renderer
const container = document.getElementById('sigma-container');
const renderer = new Sigma(graph, container, {
  renderEdgeLabels: false,
  allowInvalidContainer: false,
  settings: {
    zIndex: true,
  },
});

// Optional: zoom reset on double-click
container.addEventListener('dblclick', () => {
  renderer.getCamera().animatedReset();
});

renderer.on('clickStage', function (event) {

  graph.forEachNode((n) => {
    graph.setNodeAttribute(n, 'color', '#999')
    graph.setNodeAttribute(n, 'zIndex', 1)
    graph.setNodeAttribute(n, 'size', 10)
  });
  graph.forEachEdge((e) => graph.setEdgeAttribute(e, 'color', '#ccc'));
});

renderer.on('clickNode', ({ data, node }) => {
  const nodeId = node;

  // Reset all node and edge colors

  graph.forEachNode((n) => {
    graph.setNodeAttribute(n, 'color', '#999')
    graph.setNodeAttribute(n, 'zIndex', 1)
    graph.setNodeAttribute(n, 'size', 10)
  });
  graph.forEachEdge((e) => {
    graph.setEdgeAttribute(e, 'zIndex', 1)
    graph.setEdgeAttribute(e, 'color', '#ccc')
  });

  const visited = new Set();

  function highlightConnectedNodes(currentId, isInbound, isOutbound) {
    if (visited.has(currentId)) return;
    visited.add(currentId);

    // Outgoing edges
    if (isOutbound) {
      graph.forEachOutboundEdge(currentId, (edge, attrs, source, target) => {
        if (!visited.has(target)) {
          graph.setEdgeAttribute(edge, 'zIndex', 8888);
          graph.setNodeAttribute(target, 'zIndex', 8888)
          graph.setEdgeAttribute(edge, 'color', '#6699cc');
          graph.setNodeAttribute(target, 'color', '#6699cc');
          highlightConnectedNodes(target, false, true);
        }
      });
    }

    // Incoming edges
    if (isInbound) {
      graph.forEachInboundEdge(currentId, (edge, attrs, source, target) => {
        if (!visited.has(source)) {
          graph.setEdgeAttribute(edge, 'zIndex', 8888)
          graph.setNodeAttribute(source, 'zIndex', 8888)
          graph.setEdgeAttribute(edge, 'color', '#ff9966');
          graph.setNodeAttribute(source, 'color', '#ff9966');
          highlightConnectedNodes(source, true, false);
        }
      });
    }
  }

  // Highlight current node
  graph.setNodeAttribute(nodeId, 'size', 11)
  graph.setNodeAttribute(nodeId, 'zIndex', 9999);
  graph.setNodeAttribute(nodeId, 'color', '#000');
  highlightConnectedNodes(nodeId, true, true);

  /*
  graph.nodes().sort((a, b) => {
    return graph.getNodeAttribute(a, 'color') != '#999' - graph.getNodeAttribute(b, 'color') != '#999'
  })

  graph.edges().sort((a, b) => {
    return graph.getNodeAttribute(a, 'color') != '#ccc' - graph.getNodeAttribute(b, 'color') != '#ccc'
  })
  */
  var arr = Array.from(graph._nodes);
  graph._nodes = new Map(arr.sort(function (n1, n2) {
    if (n1[1].attributes.zIndex < n2[1].attributes.zIndex)
      return -1;
    if (n1[1].attributes.zIndex > n2[1].attributes.zIndex)
      return 1;
    return 0;
  }));

  var arr = Array.from(graph._edges);
  graph._edges = new Map(arr.sort(function (n1, n2) {
    if (n1[1].attributes.zIndex < n2[1].attributes.zIndex)
      return -1;
    if (n1[1].attributes.zIndex > n2[1].attributes.zIndex)
      return 1;
    return 0;
  }));


  renderer.refresh()
});

/*
renderer.on('leaveNode', ({ data, node }) => {
  let nodeId = node
  if(graph.getNodeAttribute(nodeId, 'color') === '#000') {
    setTimeout(() => {
      graph.setNodeAttribute(nodeId, 'zIndex', 9999);
      renderer.refresh()
    }, 100)
  }
})
*/

async function initRemote() {
  let response = await fetch('/rpc/cell%20nodes')
  let data = await response.json()
  graph.clear()
  data.nodes.forEach((node, i) => {
    graph.addNode(node.id, {
      label: node.label,
      x: 1000 * Math.cos((2 * Math.PI * i) / data.nodes.length),
      y: 1000 * Math.sin((2 * Math.PI * i) / data.nodes.length),
      size: 10,
      color: '#007acc',
      zIndex: 1,
    });
  })

  data.edges.forEach((edge, i) => {
    graph.addEdge(edge.source, edge.target, {
      color: '#ccc',
      size: 4,
      zIndex: 1,
    })
  })

  graphologyLibrary.layoutForceAtlas2.assign(graph, {
    iterations: 200,
    settings: {
      ...sensibleSettings,
      //minSpeed: 200,
      gravity: 0.5,
      scalingRatio: 10,
      strongGravityMode: false,
      edgeWeightInfluence: 3.0,
      outboundAttractionDistribution: true,
    },
  });

  renderer.refresh()
}
initRemote()


cell nodes?


In [ ]:
const path = require('path');
const gulp = require('gulp');
//const tap = require('gulp-tap');
const tasksToPromise = importer.import('gulp tasks to promise', { gulp });
const { cacheCells } = importer.import('cache notebook')
const transform = require('parallel-transform');

const NOTEBOOK_PATH = path.join(__dirname, '..');
const PROJECT_PATH = path.join(NOTEBOOK_PATH, '.notebook-graph.json')
let INTERPRET = {}
let TIMES = {}
if (fs.existsSync(PROJECT_PATH)) {
  let searchCache = JSON.parse(fs.readFileSync(PROJECT_PATH))
  INTERPRET = searchCache.searches || {}
  TIMES = searchCache.times || {}
}

const IMPORT_REGEX = /(import_notebook|importNotebook|import|interpret)\([\\\n\r\[\]\s'"]*([\s\S]*?)[\\\n\r\]\[\s'"]*\)/gi
function getImportEdges(...args) {
  let searches = args[2].split(/[\\\n\r\s'"]*,[\\\n\r\s'"]*/gi)
  let edgeCells = []
  for (let k = 0; k < searches.length; k++) {
    try {
      if (!searches[k].trim()) continue
      let cell = INTERPRET[searches[k]]
      if (typeof cell != 'undefined') {
        if (typeof cell == 'string') {
          if (cell === '') {
            INTERPRET[searches[k]] = cell = []
          } else if (typeof TIMES[cell.split('[')[0]] != 'undefined'
            && TIMES[cell.split('[')[0]] >= fs.statSync(cell.split('[')[0]).mtime
          ) {
            //console.log('Searching exact:', cell)
            INTERPRET[searches[k]] = importer.interpret(cell)
            cell = INTERPRET[searches[k]]
          }
        }
      }
      if (typeof cell != 'object') {
        //console.log('Searching:', searches[k])
        cell = importer.interpret(searches[k])
        TIMES[cell.filename] = cell.mtime
        INTERPRET[searches[k]] = cell
      }

      if (!cell || typeof cell == 'string') continue
      if (typeof cell[0] == 'undefined') {
        edgeCells.push(cell)
      } else {
        edgeCells.push(...cell)
      }
    } catch (e) {
      if (e.message.includes('Nothing found')) {
        INTERPRET[searches[k]] = ''
      }
      else {
        throw e
      }
    }
  }
  return edgeCells.flat(1)
}


function saveGraph(nodes, edges) {
  let edgesCombined = edges.map(edge => edge.source + edge.target)
  let edgesUnique = edges.filter((edge, i) => edgesCombined.indexOf(edge.source + edge.target) === i)
  let graph = {
    nodes: nodes, edges: edgesUnique, searches: Object.keys(INTERPRET).reduce((obj, key) => {
      // TODO: compensate for entire notebook includes using 'notebook.ipynb' search format
      if (typeof INTERPRET[key] == 'string') {
        obj[key] = INTERPRET[key]
      } else if (Array.isArray(INTERPRET[key])) {
        if (INTERPRET[key].length == 1) {
          obj[key] = path.join(path.dirname(INTERPRET[key][0].filename), INTERPRET[key][0].id)
        } else if (INTERPRET[key].length > 1) {
          obj[key] = INTERPRET[key][0].filename
        } else {
          obj[key] = ''
        }
      } else if (typeof INTERPRET[key].filename != 'undefined') {
        obj[key] = path.join(path.dirname(INTERPRET[key].filename), INTERPRET[key].id)
      }
      return obj
    }, {}), times: TIMES
  }
  fs.writeFileSync(PROJECT_PATH, JSON.stringify(graph, null, 4))
  return graph
}

function serveNodes() {
  let cells
  let notebooks
  let edges
  gulp.task('search notebooks', function () {
    cells = []
    edges = []
    notebooks = {}

    const addCell = (cell) => {
      if (typeof notebooks[cell.id] == 'undefined') {
        notebooks[cell.id] = {
          id: cell.id,
          label: cell.questions && cell.questions[0].length ? cell.questions[0] : (cell.markdown.match('ROUTE.*') || [])[0] || cell.markdown
        }
        cells.push(notebooks[cell.id]);
      }
    }

    return gulp.src(
      ['**/*.ipynb', '!node_modules{,/**}', '!**/\\.*'],
      { cwd: NOTEBOOK_PATH })
      .pipe(transform(10, ((file, callback) => {
        if(file.path.includes('cache')) {
          return callback()
        }
        const result = cacheCells(file.path);
        result.forEach(r => {
          if ((r.questions && r.questions[0].length) || r.markdown.includes('ROUTE')) {
            addCell(r)
            if (!r.filename.includes('cache')) {
              let m
              while ((m = IMPORT_REGEX.exec(r.code))) {
                if(m[0].includes('cache')) {
                  continue
                }
                let edgeCells = getImportEdges(...m)
                for (let i = 0; i < edgeCells.length; i++) {
                  if(edgeCells[i].filename.includes('cache')) {
                    continue
                  }
                  addCell(edgeCells[i])
                  edges.push({
                    source: r.id,
                    target: edgeCells[i].id,
                    //color:
                  })
                }
              }
            }
          }
        })
        // save in between notebooks
        saveGraph(cells, edges)
        callback()
      })));
  });

  return tasksToPromise(['search notebooks'], [])
    .then(() => saveGraph(cells, edges));
}

module.exports = serveNodes;

